## Libraries Required

In [185]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt
import pandas as pd

## Human Observed Data Preprocessing

In [ ]:
'''with open("HumanObserved-Features-Data.csv", "r") as f: #Read features csv
    x = pd.read_csv(f)
    
with open("diffn_pairs.csv", "r") as g: # Read Different pairs csv
    y = pd.read_csv(g)
    
with open("same_pairs.csv", "r") as g:  #Read same pairs csv
    z = pd.read_csv(g)
    
y = y.reindex(np.random.permutation(y.index))  # Randomize the csv file
y1 = y[0:2500]
Raw = pd.concat([y1,z],ignore_index = True)
Raw = Raw.reindex(np.random.permutation(Raw.index))

#Concatenation
df_merge = pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
df_merge=pd.merge(df_merge,x,left_on='img_id_B',right_on='img_id') 
df_merge=df_merge.drop(df_merge.columns[[0,1,3,4,14,15]],axis=1)
df_merge = df_merge.loc[:,(df_merge!=0).any(axis=0)] #Remove zero variance features
df_merge.to_csv('HumanFeatureData.csv')

#Subtraction
dfA=pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
dfB=pd.merge(Raw,x,left_on='img_id_B',right_on='img_id')
dfA=dfA.drop(dfA.columns[[0,1,2,3,4]],axis=1)
dfB=dfB.drop(dfB.columns[[0,1,2,3,4]],axis=1)
dfSub=abs(dfA-dfB)
dfSub=Raw.merge(dfSub,how='outer',left_index=True, right_index=True)
dfSub=dfSub.drop(dfSub.columns[[0,1]],axis=1)
df_Sub = df_Sub.loc[:,(df_Sub!=0).any(axis=0)] #Remove zero variance features
dfSub.to_csv('HumanFeatureDataSub.csv')'''

## GSC Data Preprocessing

In [ ]:
'''with open("GSC-Features.csv", "r") as f:  #Read features csv
    x = pd.read_csv(f)
    
with open("diffn_pairs_gsc.csv", "r") as g:  # Read Different pairs csv
    y = pd.read_csv(g)
    
with open("same_pairs_gsc.csv", "r") as g: #Read same pairs csv
    z = pd.read_csv(g)
    
y = y.reindex(np.random.permutation(y.index))  # Randomize the two csv files
z = z.reindex(np.random.permutation(z.index))
y1 = y[0:2000]
z1 = z[0:1000]  
Raw = pd.concat([y1,z1],ignore_index = True)  #Concatenate the two csv files
Raw = Raw.reindex(np.random.permutation(Raw.index))  #Randomize the csv file

#Concatenation
df_merge = pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
df_merge=pd.merge(df_merge,x,left_on='img_id_B',right_on='img_id') 
df_merge=df_merge.drop(df_merge.columns[[0,1,3,516]],axis=1)
df_merge = df_merge.loc[:,(df_merge!=0).any(axis=0)] #Remove zero variance features
df_merge.to_csv('GSCFeatureData.csv')

#Subtraction
dfA=pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
dfB=pd.merge(Raw,x,left_on='img_id_B',right_on='img_id')
dfA=dfA.drop(dfA.columns[[0,1,2,3]],axis=1)
dfB=dfB.drop(dfB.columns[[0,1,2,3]],axis=1)
dfSub=abs(dfA-dfB)
dfSub=Raw.merge(dfSub,how='outer',left_index=True, right_index=True)
dfSub=dfSub.drop(dfSub.columns[[0,1]],axis=1)
dfSub = dfSub.loc[:,(dfSub!=0).any(axis=0)]  #Remove zero variance features
dfSub.to_csv('GSCFeatureDataSub.csv')'''

## Initial Variables

In [186]:
C_Lambda = 0.01
TrainingPercent = 80 #Training split
ValPercent = 10 #Validation split
TestPercent = 10 #Test split
M = 5# Number of clusters
PHI = []

In [262]:
file = pd.read_csv('HumanFeatureData.csv') 
lenc = len(file.columns) #Obtain Number of features
lenc

20

## Generate Raw Data

In [263]:
def GenerateRawData(file_path): #Function to generate Feature matrix and Target vector
    data_target = []
    data_input = []
    x = pd.read_csv(file_path)
    data_target = x['target'].tolist()
    
    data_input = x.iloc[:,2:lenc]
    
    data = np.array(data_input).T
    target = data_target

    return data,target

## Functions Used

In [264]:

def GenerateBigSigma(Data, MuMatrix,TrainingPercent): #Generating the Covariance matrix
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
        
    return BigSigma

def GetScalar(DataRow,MuRow, BigSigInv): #This function is used to calculate the Gaussian basis function
    #(Without taking the exponential) on the input vector and in the end it returns a scalar value.
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

def GetRadialBasisOut(DataRow,MuRow, BigSigInv): #This function takes the value from the above function gets the 
    #final scalar value for the Gaussian basis function by taking the exponential.    
    phi_x = math.exp(-GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80): # This function creates the design matrix 
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    return PHI

def GetWeightsClosedForm(PHI, T, Lambda): #This function calculates the weights by performing the matrix multiplication
    #on the Moore-penrose psuedo inverse of the design matrix and the vector for target variables.
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    return W

def GetValTest(VAL_PHI,W): #This function performs matrix multiplication between the weight vector and the design matrix
    Y = np.dot(W,np.transpose(VAL_PHI))
    return Y

def GetErms(VAL_TEST_OUT,ValDataAct): #This function calculates the root mean square error and accuracy.
    sum = 0.0
    accuracy = 0.0
    counter = 0
    
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT))) #Calculating the accuracy
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT)))) # Returning the accuracy and the RMS error.

## Fetch and Prepare Dataset

In [274]:
RawData,RawTarget = GenerateRawData('HumanFeatureData.csv') 

## Prepare Training Data

In [275]:
TrainingLen = int(math.ceil(len(RawTarget)*(TrainingPercent*0.01)))
TrainingTarget = np.array(RawTarget[:TrainingLen])

T_len = int(math.ceil(len(RawData[0])*0.01*TrainingPercent))
TrainingData = np.array(RawData[:,0:T_len])

## Prepare Validation Data

In [276]:
valSize = int(math.ceil(len(RawData[0])*ValPercent*0.01))
V_End = len(TrainingTarget) + valSize
ValData = np.array(RawData[:,len(TrainingTarget)+1:V_End])  

ValDataAct = np.array(RawTarget[len(TrainingTarget)+1:V_End])
print(ValDataAct.shape)
print(ValData.shape)

(329,)
(18, 329)


## Prepare Test Data

In [277]:
TestSize = int(math.ceil(len(RawData[0])*TestPercent*0.01))
T_End = len(TrainingTarget) + len(ValDataAct) + valSize
TestData = np.array(RawData[:,len(TrainingTarget) + len(ValDataAct):T_End])

TestDataAct = np.array(RawTarget[len(TrainingTarget) + len(ValDataAct):T_End])
print(TestDataAct.shape)
print(TestData.shape)

(329,)
(18, 329)


## Closed Form Solution [Finding Weights using Moore- Penrose pseudo- Inverse Matrix]

In [278]:
ErmsArr = []
AccuracyArr = []


kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData)) #K-means clustering
Mu = kmeans.cluster_centers_ #Centroids of each cluster

BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent) #covariance matrix
TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent) # get the training design matrix
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) # get the updated weights
TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) #Testing design matrix
VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100) #Validation design matrix

## Finding Erms on training, validation and test set 

In [279]:
TR_TEST_OUT  = GetValTest(TRAINING_PHI,W) #Matrix multiplication between Training design matrix and weight vector
VAL_TEST_OUT = GetValTest(VAL_PHI,W) #Matrix multiplication between Validation design matrix and weight vector
TEST_OUT     = GetValTest(TEST_PHI,W) #Matrix multiplication between Testing design matrix and weight vector


TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))#Training Erms
ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))#Validation Erms
TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))#Testing Erms

In [280]:
print ('UBITname      = SHEERSHA')
print ('Person Number = 50291476')
print ("-------Closed Form with Radial Basis Function-------")
print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))
print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))
print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))

UBITname      = SHEERSHA
Person Number = 50291476
-------Closed Form with Radial Basis Function-------
E_rms Training   = 0.47801690289811866
E_rms Validation = 0.4927322539042101
E_rms Testing    = 0.5702167038492976


## Gradient Descent solution for Linear Regression

In [281]:
W_Now        = np.dot(220, W) 
learningRate = 0.05
La = 5
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []

for i in range(0,2400):
    
    #print ('---------Iteration: ' + str(i) + '--------------')
    #This part of the code updates the weights after each iteration using Stochastic Gradient Descent
    Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i]) #This 
    #line calculates the first part of delta E equation which is added to the regularization term
    La_Delta_E_W  = np.dot(La,W_Now) #Regularization term
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W) #We get Delta E by adding the above two terms    
    Delta_W       = -np.dot(learningRate,Delta_E) #This is the term which updates the weights
    W_T_Next      = W_Now + Delta_W #Updating of the weights
    W_Now         = W_T_Next
    
    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)  #Calculating Training Erms after each iteration
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next)  #Calculating Validation Erms after each iteration
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) #Calculating Testing Erms after each iteration
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

In [282]:
print ('----------Gradient Descent Solution--------------------')
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Gradient Descent Solution--------------------
E_rms Training   = 0.47496
E_rms Validation = 0.49219
E_rms Testing    = 0.5615
